# Direct and inverse geometry of 3d robots
This notebook introduces the kinematic tree of Pinocchio for a serial manipulator, explain how to compute the forward and inverse geometry (from configuration to end-effector placements, and inversely). The ideas are examplified with a simplified case-study taken from parallel robotics.

In [ ]:
import magic_donotload

## Set up
We will need Pinocchio, Gepetto-Viewer, SciPy for the solvers

In [7]:
import math
import time
import numpy as np
from numpy.linalg import norm,inv,pinv,svd,eig
import pinocchio as pin
import example_robot_data as robex
from scipy.optimize import fmin_bfgs,fmin_slsqp

## Kinematic tree in Pinocchio
Let's now play with 3D robots. We will load the models from URDF files.

*The robot UR5* is a low-cost manipulator robot with good performances. It is a fixed robot with one 6-DOF arms developed by the Danish company Universal Robot. All its 6 joints are revolute joints. Its configuration is in R^6 and is not subject to any constraint. The model of UR5 is described in a URDF file, with the visuals of the bodies of the robot being described as meshed (i.e. polygon soups) using the Collada format ".dae". Both the URDF and the DAE files are available in the repository in the model directory. 

This robot model, as well as other models used in the notebooks, are installed from the apt paquet robotpkg-example-robot-data and stored in /opt/openrobots/share/example-robot-data.


In [2]:
# %load -r 9 tp2/simple_pick_and_place 
robot = robex.load('ur5')

In [5]:
from pinocchio.robot_wrapper import RobotWrapper
my_robot = RobotWrapper.BuildFromURDF('schunk.urdf', '.', None)

The kinematic tree is represented by two C++ objects called Model (which contains the model constants: lengths, masses, names, etc) and Data (which contains the working memory used by the model algorithms). Both C\++ objects are contained in a unique Python class. The first class is called RobotWrapper and is generic.

In [3]:
print(my_robot.data)

For the next steps, we are going to work with the RobotWrapper.

Import the class RobotWrapper and create an instance of this class in the python terminal. At initialization, RobotWrapper will read the model description in the URDF file given as argument. In the following, we will use the model of the UR5 robot, available in the directory "models" of pinocchio (available in the homedir of the VBox). The code of the RobotWrapper class is in /opt/openrobots/lib/python2.7/site-packages/pinocchio/robot_wrapper.py . Do not hesitate to have a look at it and to take inspiration from the implementation of the class functions.

Here are some import methods of the class.


* robot.q0 contains a reference initial configuration of the robot (not a pretty good one for the UR-5).

* robot.index('joint name') returns the index of the joint.

In [ ]:
robot.index('wrist_3_joint')

* robot.model.names is a container (~list) that contains all the joint names

In [ ]:
for i,n in enumerate(robot.model.names): print(i,n)

* robot.model.frames contains all the import frames attached to the robot. 

In [ ]:
for f in robot.model.frames:
    print(f.name,'attached to joint #',f.parent)

* robot.placement(idx) and robot.framePlacement(idx) returns the placement (i.e. translation+rotation of the joint / frame in argument.

In [ ]:
robot.placement(robot.q0,6) # Placement of the end effector.
print(robot.q0)

The dimension of the configuration space (i.e. the number of joints) is given in:

In [2]:
NQ = robot.model.nq
NV = robot.model.nv   # for this simple robot, NV==NQ

NameError: name 'robot' is not defined

## Display simple geometries
The robot is displayed in the viewer. We are going to use Meshcat to visualize the 3d robot and scene. First open the viewer and load the robot geometries.

In [8]:
from tp2.meshcat_viewer_wrapper import *
my_robot = RobotWrapper.BuildFromURDF('schunk.urdf', '.', None)
viz = MeshcatVisualizer(my_robot)
q = np.array([0., 1.57, 0, 0, 0, 0])

viz.display(q)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


In [9]:
viz.viewer.jupyter_cell()

In [7]:
q = qopt.copy()
#q[4] = -1.7
#q[1] = -1.7
#q[2] = math.pi+q[1]-q[4]
#q[3] = 0
#q[0] = 0
#-0.20 => pi/2+q,pi/2
print(q)
viz.display(q)
print(cost(qopt))
print(constraint_9(qopt))
print(cost(q))
print(Mtarget)
print(my_robot.placement(q,6))
#print(norm(my_robot.placement(q,6).rotation - Mtarget.rotation))
#print(my_robot.placement(q,6))

[-0.13740908 -0.54854748  1.78463938  0.          0.80840579  0.        ]
9.856237486885341e-07
[0. 0.]
9.856237486885341e-07
  R =
-1  0  0
 0  1  0
 0  0 -1
  p = -0.4 0.05 0.99

  R =
  -0.990575     0.13697 2.02592e-08
    0.13697    0.990575 1.46586e-07
9.60565e-12 1.47979e-07          -1
  p = -0.400001 0.0499992      0.99



In [1]:
IDX_TOOL = my_robot.model.getFrameId('frametool')

Mtarget = pin.XYZQUATToSE3([-0.4, 0.05, 0.99, 1, 0, 0, 0])
Mtarget.rotation[0][0] = -1
Mtarget.rotation[1][1] = 1
Mtarget.rotation[2][2] = -1

# %load -r 36-62 tp3/inverse_kinematics.py
q = np.array([0, 0, 0, 0, 0, 0])
herr = [] # Log the value of the error between tool and goal.
# Loop on an inverse kinematics for 200 iterations.
for i in range(1):  # Integrate over 2 second of robot life

    # Run the algorithms that outputs values in robot.data
    pin.framesForwardKinematics(my_robot.model,my_robot.data,q)
    pin.computeJointJacobians(my_robot.model,my_robot.data,q)

    # Placement from world frame o to frame f oMtool
    print(len(my_robot.data.oMf))
    print(IDX_TOOL)
    #oMtool = my_robot.data.oMf[IDX_TOOL-1]

    # 3D jacobian in world frame
    #o_Jtool3 = pin.computeFrameJacobian(my_robot.model,my_robot.data,q,IDX_TOOL,pin.LOCAL_WORLD_ALIGNED)[:3,:]

    # vector from tool to goal, in world frame
    #o_TG = oMtool.translation-oMgoal.translation
    
    # Control law by least square
    #vq = -pinv(o_Jtool3)@o_TG

    #q = pin.integrate(my_robot.model,q, vq * DT)
    #viz.display(q)
    #time.sleep(1e-3)

    #herr.append(o_TG) 

NameError: name 'my_robot' is not defined

A configuration *q* can be displayed in the viewer:

In [10]:
q = np.array([0, 0, 0, 0.0, -0.0, 0.])

viz.display(q)

tableID = "world/table"
viz.addBox(tableID, [1, 0.5, 0.1], colors.green)
q_table = [-0.4, 0.15, 0.7, 1, 0, 0, 0]
viz.applyConfiguration (tableID, q_table)

feets = ['world/feet%s'%i for i in range(4)]
qs = [[0.05, -0.85], [0.35, -0.05]]
qs_f = [[qs[0][i%2], qs[1][i//2], 0.325, 1, 0, 0, 0] for i in range(4)]
for i in range(4):
    viz.addBox(feets[i], [0.1, 0.1, 0.65], colors.blue)
    viz.applyConfiguration (feets[i], qs_f[i])

cylinders = ['world/cylinder%s'%i for i in range(9)]
qs = [[-0.4, -0.6, -0.5], [0.05, 0.15, 0.25]]
qs_c = [[qs[0][i%3], qs[1][i//3], 0.8, 0.707, 0, 0, 0.707] for i in range(9)]
for i in range(9):
    viz.addCylinder(cylinders[i], 0.065, 0.045, colors.red)
    viz.applyConfiguration (cylinders[i], qs_c[i])

charriot = "world/charriot"
viz.addBox(charriot, [0.5, 0.25, 0.3], colors.black)
q = [0, -0.5, 0.15, 1, 0, 0, 0]
viz.applyConfiguration (charriot, q)

cylinders_out = ['world/cylinder_out%s'%i for i in range(9)]
qs = [[-0.2, -0.1, 0, 0.1, 0.2, 0.3], [-0.45, -0.55]]
qs_o = [[qs[0][i%5], qs[1][i//5], 0.375, 0.707, 0, 0, 0.707] for i in range(9)]
for i in range(9):
    viz.addCylinder(cylinders_out[i], 0.15, 0.045, colors.white)
    viz.applyConfiguration (cylinders_out[i], qs_o[i])


In [ ]:
q = np.array([0, 1.75, 0.4, 0, 0, 0])
viz.display(q)
Mref = pin.XYZQUATToSE3([0, 0, 1.69, 0.707, 0, 0, -0.707])
print(q0)
rMcyl_ref = my_robot.placement(q0,6).inverse()*Mref
o_box = my_robot.placement(q, 6) * rMcyl_ref
viz.applyConfiguration(cylinder_test, o_box)


In [7]:
q = np.array([0, 0, 0, 0.0, -0.0, 0.])
viz.display(q)
print(my_robot.placement(q,6))
cylinder_test = 'world/test'
q = [0, 0, 1.69, 0.707, 0, 0, -0.707]
viz.addCylinder(cylinder_test, 0.1, 0.045, colors.green)
viz.applyConfiguration(cylinder_test, qs_c[2])

  R =
           1  7.96077e-06 -1.94944e-11
-7.96077e-06            1  9.14043e-16
 1.94944e-11   1.0346e-16            1
  p = -2.79376e-08  -0.00526171        1.557



In [6]:
def cost(q):
    '''Compute score from a configuration''' 
    M = my_robot.placement(q,6)
    #rMcyl = M.inverse() * Mtarget
    #return norm(pin.log(rMcyl.inverse() * rMcyl_ref).vector)
    #return norm(rMcyl.inverse()*rMcyl_ref)
    cost = 0
        
    #cost += 10*norm(M.translation[2] - Mtarget.translation[2])**2
    cost += norm((M.translation-Mtarget.translation))
    return cost

def constraint_9(ps):
    res = np.zeros(2)
    res[0] = ps[2] - (math.pi+ps[1]-ps[4])
    res[1] = ps[3]
    return res

def callback(q):
    #viz.applyConfiguration('world/box', Mtarget)
    viz.display(q)
    time.sleep(1e-1)
    
print("begin")

#Mtarget = pin.SE3(pin.utils.rotate('x',3.14/4), np.array([0.5, 0.1, 0.2]))  # x,y,z
q0 = np.array([0, 0, 0, 0, 0, 0])
Mtarget = pin.XYZQUATToSE3([-0.4, 0.05, 0.99, 1, 0, 0, 0])
Mtarget.rotation[0][0] = -1
Mtarget.rotation[1][1] = 1
Mtarget.rotation[2][2] = -1
#Mtarget = pin.XYZQUATToSE3([0, -0.45, 0.53, 1, 0, 0, 0])
#Mtarget = pin.XYZQUATToSE3(qs_c[0])
#Mref = pin.XYZQUATToSE3([0, -0.00526171, 1.69, 0.707, 0, 0, -0.707])
#rMcyl_ref = my_robot.placement(q0,6).inverse()*Mref
#qopt = fmin_bfgs(cost, q0, callback=callback)
qopt = fmin_slsqp(cost, q0, callback=callback, f_eqcons=constraint_9, iprint=2, full_output=1)[0]
print(qopt)

begin
  NIT    FC           OBJFUN            GNORM
    1     8     6.960911E-01     4.152150E-01
    2    16     2.431307E-01     3.650117E-01
    3    24     1.791870E-01     2.879287E-01
    4    33     1.131647E-01     3.446176E-01
    5    41     8.220350E-02     3.253894E-01
    6    50     3.798079E-02     3.532585E-01
    7    58     2.950216E-02     3.074451E-01
    8    67     1.623700E-02     3.474329E-01
    9    76     1.143822E-02     2.996014E-01
   10    85     5.122701E-03     3.863500E-01
   11    95     3.022797E-03     4.096974E-01
   12   104     1.898166E-03     2.448459E-01
   13   113     1.470561E-03     4.207197E-01
   14   122     7.648917E-04     3.732845E-01
   15   131     4.615870E-04     3.226589E-01
   16   140     2.423536E-04     4.056499E-01
   17   149     2.050799E-04     4.014516E-01
   18   158     1.292947E-04     3.387074E-01
   19   167     7.617984E-05     3.656130E-01
   20   176     3.300250E-05     4.061624E-01
   21   186     1.914050E-05

In [11]:
q = np.array([0, 0, 0, 0.0, -0.0, 0.])
print(cost(q))
sign = lambda x: (abs(x)-math.pi > 0) -0
sign(1.57)


AttributeError: 'SE3' object has no attribute 'target'

In [50]:
print(Mref)
test = my_robot.placement(np.array([0,0,0,0,0,0]),6).inverse()*Mref
#print(my_robot.placement(np.array([0,0,0,0,0,0]),6))
print(test)
test1 = my_robot.placement(qopt,6).inverse()*Mtarget
print(test1)
print(test1.inverse()*test)
print(pin.log(test1.inverse()*test).vector)
#print(norm(test1.inverse()*test))
#print(rMcyl_ref)
#print(norm(pin.log(test.inverse()*rMcyl_ref).vector))
#print(cost(qopt))
print(qopt)
rMref = my_robot.placement(qopt,6).inverse()*Mref
print(norm(pin.log(rMref.inverse()*test).vector))
print(Mref)

  R =
        1         0         0
        0  0.000302  0.999698
        0 -0.999698  0.000302
  p =           0 -0.00526171        1.69

  R =
           1 -2.42364e-09 -7.95837e-06
 7.96077e-06     0.000302     0.999698
-1.94944e-11    -0.999698     0.000302
  p = 2.79402e-08 4.37836e-09       0.133

  R =
           1 -2.44714e-08 -1.21869e-08
 2.44714e-08            1  7.07933e-09
 1.21869e-08 -7.07933e-09            1
  p =  7.58108e-08  2.09914e-08 -1.16511e-08

  R =
           1 -1.45994e-08  -7.9339e-06
  7.9363e-06  0.000302007     0.999698
-1.22063e-08    -0.999698  0.000302007
  p = -4.62498e-08 -1.75546e-08        0.133

[ 2.54030823e-07 -1.04406360e-01  1.04469413e-01 -1.57002010e+00
 -6.22048565e-06  6.24341974e-06]
[-0.08919517 -0.8759631   1.39313552 -3.14159429 -0.87249404 -0.08919243]
0.0276624741227777
  R =
        1         0         0
        0  0.000302  0.999698
        0 -0.999698  0.000302
  p =           0 -0.00526171        1.69



Other geometries (cubes, spheres, etc) can be displayed as well.

In [ ]:
# %load -r 15-23 tp2/simple_pick_and_place.py

# Add a red box in the viewer
ballID = "world/ball"
viz.addSphere( ballID, 0.5, colors.red )

# Place the ball at the position ( 0.5, 0.1, 0.2 )
# The viewer expect position and rotation, apppend the identity quaternion
q_ball = [0, 0, 0, 1, 0, 0, 0]
viz.applyConfiguration( ballID, q_ball )

## Forward (direct) geometry

First, let's do some forward geometry, i.e. use Pinocchio to compute where is the end effector knowning the robot configuration.

### Simple pick ...

Say we have a target at position [.5,.1,.2] and we would like the robot to grasp it.
First decide (by any way you want, e.g. trial and error) the configuration of the robot so that the end effector touches the ball.


In [ ]:
# %load -r 29-36 tp2/simple_pick_and_place.py

q0 = np.zeros(NQ)  ### set the correct values here
q0[0] = -0.1
q0[1] = -1.45
q0[2] = 1.8
q0[3] = .4
q0[4] = 1.5
q0[5] = 1
viz.display(q0)
print(q0)

# Take care to explicitely mention copy when you want a copy of array.
q = q0.copy()

### ... and simple place

At the reference position you built, the end effector placement can be obtained by calling 

In [ ]:
robot.placement(q,6).translation

q0 = np.zeros(NQ)
while (norm(robot.placement(q0,6).translation - [.5,.1,.2]) >= 0.2):
    q0 = (np.random.rand(NQ)-0.5)*2*1.7
    viz.display(q0)

Only the translation part of the placement has been selected. The rotation is free.

Now, choose any trajectory you want in the configuration space (it can be sinus-cosinus waves, polynomials, splines, straight lines). Make a for loop to display the robot at sampling positions along this trajectory. The function sleep can be used to slow down the loop.

In [ ]:
time.sleep(.1) # in second

At each instant of your loop, recompute the position of the ball and display it so that it always "sticks" to the robot end effector.

In [ ]:
## TODO ####################################################
#### Replace here with your initial configuration
NQ = 6
q0 = np.zeros(NQ)  ### set the correct values here
q0[0] = -0.1
q0[1] = -1.45
q0[2] = 1.8
q0[3] = .4
q0[4] = 1.5
q0[5] = 1
q = q0# = np.random.rand(NQ)*6-3  
### TODO ####################################################

# Compute initial translation between effector and box.
# Translation of end-eff wrt world at initial configuration
o_eff = robot.placement(q, 6).translation
# Translation of box wrt world
o_ball = q_ball[:3]
eff_ball = o_ball - o_eff
o_ball = robot.placement(q,6)*eff_ball
#viz.applyConfiguration(ballID, o_ball.tolist()+[1,0,0,0])
#viz.display(q)
time.sleep(0.1)

for i in range(0):
    ### Replace here by your choice of computing q(t)
    q += [0,0,-math.pi/200,0,0,0]
    print(q)
    ### TODO ####################################################
    ### Replace here by your computation of the new box position 
    o_ball = robot.placement(q,6)*eff_ball
    
    ### /TODO ###################################################
    
    # Display the new robot and box configurations.
    # The viewer expect a placement (position-rotation).
#    viz.applyConfiguration(ballID, o_ball.tolist()+[1,0,0,0])
#    viz.display(q)
    time.sleep(0.1)


The solution is below, should you need it.

In [ ]:
# %load -r 46-69 tp2/simple_pick_and_place.py
q0 = np.zeros(NQ)  ### set the correct values here
q0[0] = -0.1
q0[1] = -1.45
q0[2] = 1.8
q0[3] = .4
q0[4] = 1.5
q0[5] = 1
q = q0.copy()
q_ball = [0.5, 0.1, 0.2, 1, 0, 0, 0]
viz.applyConfiguration( ballID, q_ball )
viz.display(q0)
#time.sleep(10)

# Random velocity of the robot driving the movement
vq = np.array([ 2.,0,0,4.,0,0])

idx = robot.index('wrist_3_joint')
print(idx)
o_eff = robot.placement(q, idx).translation  # Position of end-eff wrt world at current configuration

o_ball = q_ball[:3]                          # Position of ball wrt world
eff_ball = o_ball - o_eff                    # Position of ball wrt eff

for i in range(50):
    # Chose new configuration of the robot
    q += vq / 40
    q[2] = 1.71 + math.sin(i * 0.05) / 2

    # Gets the new position of the ball
    o_ball = robot.placement(q, idx) * eff_ball

    # Display new configuration for robot and ball
    viz.applyConfiguration(ballID, o_ball.tolist()+[0,0,0,1])
    viz.display(q)
    time.sleep(0.1)


### Pick and place in 3D

Say now that the object is a rectangle and not a sphere. Pick the object at a reference position with the rotation that is imposed, so that the end effector is aligned with one of the faces of the rectangle.


In [ ]:
# %load -r 77-85 tp2/simple_pick_and_place.py
# Add a red box in the viewer
boxID = "world/box"
viz.delete( ballID )
viz.addBox( boxID, [0.1, 0.2, 0.1], colors.magenta )

# Place the box at the position ( 0.5, 0.1, 0.2 )
q_box = [0.5, 0.1, 0.2, 1, 0, 0, 0]
viz.applyConfiguration( 'world/box', q_box )


A configuration with the arm nicely attached to the box is:

In [ ]:
# %load -r 30-37 tp2/simple_pick_and_place.py
q0 = np.zeros(NQ)
q0[0] = -0.375
q0[1] = -1.2
q0[2] = 1.71
q0[3] = -q0[1] - q0[2]
q0[4] =  q0[0]

q_box = [0.5, 0.1, 0.2, 1, 0, 0, 0]
viz.applyConfiguration( 'world/box', q_box )
viz.viz.display(q0)
q = q0.copy()

Redo the same question as before, but now also choosing the orientation of the box. For that, at each robot configuration in your for-loop, compute the box placement wrt the world (let's denote it by oMbox) and display both the box and the robot configuration in the view.

In [ ]:
%do_not_load -r 91- tp2/simple_pick_and_place.py

q = q0.copy()
# Random velocity of the robot driving the movement
vq = np.array([ 2.,0,0,4.,0,0])

idx = robot.index('wrist_3_joint')
oMeff = robot.placement(q, idx)  # Position of end-eff wrt world at current configuration

oMbox = pin.XYZQUATToSE3(q_box)              # Position of box wrt world
effMbox = oMeff.inverse()*oMbox              # Position of box wrt eff

for i in range(50):
    # Chose new configuration of the robot
    q += vq / 40
    q[2] = 1.71 + math.sin(i * 0.05) / 2

    # Gets the new position of the ball
    o_box = robot.placement(q, idx) * effMbox

    # Display new configuration for robot and ball
    viz.applyConfiguration(boxID, o_box)
    viz.display(q)
    time.sleep(0.1)


## Inverse geometry

We only yet computed the forward geometry, i.e. from configurations to end-effector placement. Let's to the inverse map not.

### Inverse geometry in 3D

Let's now first control the position (i.e. translation only) of the end effector of a manipulator robot to a given position. For this first part, we will use the fixed serial-chain robot model.

Recall first that the position (3D) of the joint with index "i=6" at position "q" can be access by the following two lines of code.

In [ ]:
robot.placement(q,6).translation

def cost(ps):
    return norm(robot.placement(ps,6).translation- q_box[:3])

def callback(q):
    viz.display(q)
    time.sleep(.5)
    
x0 = np.zeros(NQ)
xopt_bfgs = fmin_bfgs(cost, x0, callback=callback)
print('\n *** Xopt in BFGS = %s \n\n\n\n' % xopt_bfgs)
print(cost(xopt_bfgs))
viz.display(xopt_bfgs)
print(cost(q0))

In [ ]:
print(robot.placement(q,6).translation)
print(q_box[:3])

Using the scipy solver [used in the previous notebook](1_geometry_2d.ipynb#section_optim), compute a configuration q where the end effector reaches p. For that, implement a cost function that takes a configuration as argument and returns the squared distance between the end effetor and the target.

In [ ]:
# %load -r 35- tp2/invgeom3d.py
viz.addSphere("world/ball", .05, 'green')
viz.addBox("world/blue", [.08]*3, [.2, .2, 1., .5])

#
# OPTIM 3D #########################################################
#

def cost(q):
    '''Compute score from a configuration'''
    p = robot.placement(q, 6).translation
    return norm(p - target)**2

def callback(q):
    viz.applyConfiguration('world/ball', target.tolist()+[0,1,0,0])
    viz.applyConfiguration('world/blue', robot.placement(q, 6))
    viz.display(q)
    time.sleep(1e-1)

target = np.array([0.5, 0.1, 0.2])  # x,y,z
qopt = fmin_bfgs(cost, robot.q0, callback=callback)


### Inverse geometry in 6D
6D means: translation and rotation. Change the previous cost function for a cost measuring the difference between the current placement root.placement(q,6) and a reference placement oMdes. 
For that, you can use the SE(3) log function to score the distance between two placements. The log returns a 6D velocity, represented by a class Motion, that must be transformed to a vector of R^6 from which you can take the norm.


In [ ]:
pin.log(pin.SE3.Identity()).vector

In [ ]:
# %load -r 35- tp2/invgeom6d.py
viz.addBox("world/box",  [.05,.1,.2], [1., .2, .2, .5])
viz.addBox("world/blue", [.05,.1,.2], [.2, .2, 1., .5])

#
# OPTIM 6D #########################################################
#

def cost(q):
    '''Compute score from a configuration'''
    M = robot.placement(q, 6)
    return norm(pin.log(M.inverse() * Mtarget).vector)

def callback(q):
    viz.applyConfiguration('world/box',  Mtarget)
    viz.applyConfiguration('world/blue', robot.placement(q, 6))
    viz.display(q)
    time.sleep(1e-1)

Mtarget = pin.SE3(pin.utils.rotate('x',3.14/4), np.array([0.5, 0.1, 0.2]))  # x,y,z
qopt = fmin_bfgs(cost, robot.q0, callback=callback)

print('The robot finally reached effector placement at\n',robot.placement(qopt,6))


### Optimizing in the quaternion space

Let's now work with a floating robot: the quadruped ANYmal. This robot has 12 joints, but Q-space of size 19 (robot.model.nq) and Q-tangent space of size 18 (robot.model.nv). This is because with need 7D vector to encode the robot placement in space, which indeed to only 6 DOF.


In [ ]:
robot = robex.load('solo12')
viz = MeshcatVisualizer(robot)
viz.viewer.jupyter_cell()

In [ ]:
from math import *
q1 = robot.q0.copy()
#q1[0:3] = qopt[0:3]
#q1[4] = qopt[4]
q1[4] = 0
#q1[7:10] = qopt[7:10]
#q1[10:13] = qopt[10:13]
#q1[13:16] = qopt[13:16]
#q1[16:] = qopt[16:]
viz.display(q1)
robot.feetIndexes = [ robot.model.getFrameId(frameName) for frameName in ['HR_FOOT','HL_FOOT','FR_FOOT','FL_FOOT' ] ]
print(norm(qopt[3:7]))
print(acos(qopt[3])*2)
#print(len(robot.q0))
#print(robot.q0)

In [ ]:
p_i = pin.SE3ToXYZQUAT(robot.framePlacement(qopt, robot.feetIndexes[2]))
p_i[2] = 0
viz.applyConfiguration('world/%s'%colors[0], p_i)
print(p_i)
print(norm(robot.framePlacement(qopt, robot.feetIndexes[2]).rotation))

In [ ]:
cost = 0.
for i in range(4):
    p_i = robot.framePlacement(qopt, robot.feetIndexes[i])
    print(p_i.translation)
    print(targets[i])
    cost += norm(p_i.translation-targets[i])**2
print(cost)
print(norm(qopt[3:7]))


Run the following code. Can you explain what just happened? Then correct it to have a proper optimization of ANYmal configuration.

In [ ]:
# %load -r 28- tp2/floating.py

#robot.feetIndexes = [ robot.model.getFrameId(frameName) for frameName in ['HR_FOOT','HL_FOOT','FR_FOOT','FL_FOOT' ] ]

# --- Add box to represent target
colors = ['red','blue','green','magenta']
for color in colors:
    viz.addSphere("world/%s"%color, .05, color)
    viz.addSphere("world/%s_des"%color, .05, color)

#
# OPTIM 6D #########################################################
#

targets = [
    np.array(  [-0.7, -0.2,  1.2]),
    np.array(  [-0.3,  0.5,  0.8]),
    np.array(  [ 0.3,  0.1, -0.1]),
    np.array(  [ 0.9,  0.9,  0.5])
    ]

for i in range(4):
    targets[i] = robot.framePlacement(qopt, robot.feetIndexes[i]).translation
    
#targets = [
#    np.array(  [-0.3141204, 0.06255256,1.7936871 ]),
#    np.array(  [-0.3,  0.5,  0.8]),
#    np.array(  [0.28843421, 0.15301477, 1.25817612]),
#    np.array(  [0.52568868, 0.58443093, 1.54813337])
#    ]
for i in range(4): 
    targets[i][2]+=1
   # viz.applyConfiguration('world/%s_des'%colors[i], list(targets[i])+[1,0,0,0])

def cost(q):
    '''Compute score from a configuration'''
    cost = 0.
    for i in range(4):
        p_i = robot.framePlacement(q, robot.feetIndexes[i])
        cost += norm(p_i.translation-targets[i])**2# + 10*(norm(q[3:7])- 1)**2
    return cost

def callback(q):
    #viz.applyConfiguration('world/box', Mtarget)

    for i in range(4):
        p_i = robot.framePlacement(q, robot.feetIndexes[i])
        viz.applyConfiguration('world/%s'%colors[i], p_i)
        viz.applyConfiguration('world/%s_des'%colors[i], list(targets[i])+[1,0,0,0])

    viz.display(q)
    time.sleep(1e-1)
    
print("begin")

#Mtarget = pin.SE3(pin.utils.rotate('x',3.14/4), np.array([0.5, 0.1, 0.2]))  # x,y,z
qopt = fmin_bfgs(cost, robot.q0, callback=callback)



## Configuration of parallel robots
A parallel robot is composed of several kinematic chains (called the robot legs) that are all attached to the same end effector. This imposes strict constraints in the configuration space of the robot: a configuration is valide iff all the legs meets the same end-effector placement. We consider here only the geometry aspect of parallel robots (additionnally, some joints are not actuated, which causes additional problems).

The kinematic structure of a paralel robot indeed induces loops in the joint connection graph. In Pinocchio, we can only represents (one of) the underlying kinematic tree. The loop constraints have to be handled separately. An example that loads 4 manipulator arms is given below.

In [ ]:
import tp2.load_ur5_parallel as robex2
robot = robex2.load()

In [ ]:
viz = MeshcatVisualizer(robot)
viz.viewer.jupyter_cell()

In [ ]:
viz.display(robot.q0)

In [ ]:
[w, h, d] = [0.5, 0.5, 0.005]
color = [red, green, blue, transparency] = [1, 1, 0.78, .8]
viz.addBox('world/robot0/toolplate', [w, h, d], color)
Mtool = pin.SE3(pin.utils.rotate('z', 1.268), np.array([0, 0, .75]))
viz.applyConfiguration('world/robot0/toolplate', Mtool)

The 4 legs of the robot are loaded in a single robot model. The 4 effector placements are computed by:

In [ ]:
effIdxs = [ robot.model.getFrameId('tool0_#%d' % i) for i in range(4)]
robot.framePlacement(robot.q0,effIdxs[0])

In [ ]:
from math import sqrt
oMrs = [robot.framePlacement(robot.q0,effIdxs[i]) for i in range(4)]
r0Mr1 = oMrs[0].inverse()*oMrs[1]
r1Mr2 = oMrs[1].inverse()*oMrs[2]
r2Mr3 = oMrs[2].inverse()*oMrs[3]
r3Mr0 = oMrs[3].inverse()*oMrs[0]
rsMbox = [oMrs[i].inverse()*Mtool for i in range(4)]

The loop constraints are that the relative placement of every leg end-effector must stay the same that in the initial configuration given as example in with the configuration *robot.q0* and the plate placement *Mtool*. To be valid, a configuration *q* must satisfy these 4 relative placement constraints.

Consider now that the orientation of the tool plate is given by the following quaternion, with the translation that you like (see [the notebook about rotations if you need more details](appendix1_quaternions.ipynb)): 



In [ ]:
from pinocchio import SE3, Quaternion
quat = pin.Quaternion(0.7,0.2,0.2,0.6).normalized()
print(quat.matrix())
R  = quat.matrix()                # Create a rotation matrix from quaternion
p  = np.array([0.,0.,0.75])     # Translation (R3) vector)
M  = SE3(R,p)
viz.applyConfiguration('world/robot0/toolplate', M)

In [ ]:
def cost(q):
    '''Compute score from a configuration'''
    cost = 0
    oMrs = [robot.framePlacement(q,effIdxs[i]) for i in range(4)]
    #cost += 100*norm(pin.log((oMrs[0].inverse() * oMrs[1]).inverse() * r0Mr1).vector)**2
    #cost += 100*norm(pin.log((oMrs[1].inverse() * oMrs[2]).inverse() * r1Mr2).vector)**2
    #cost += 100*norm(pin.log((oMrs[2].inverse() * oMrs[3]).inverse() * r2Mr3).vector)**2
    #cost += 100*norm(pin.log((oMrs[3].inverse() * oMrs[0]).inverse() * r3Mr0).vector)**2
    #cost = 0
    for i in range(4):
        cost += norm(pin.log((oMrs[i].inverse() * Mtarget).inverse() * rsMbox[i]).vector)
        #cost += norm(pin.log(oMrs[i].inverse()*Mtarget).vector)
    return cost

def callback(q):
    viz.display(q)
    time.sleep(1e-1)

Mtarget = SE3(R,p)
qopt = fmin_bfgs(cost, robot.q0, callback=callback)

print('The robot finally reached effector placement at\n',robot.placement(qopt,6))

In [ ]:
#Mtarget = Mtool
print(cost(qopt))